In [ ]:
# Imports
import sys

sys.path.insert(0, "../..")

from qiskit import QuantumCircuit, qasm2
from qiskit.quantum_info import Clifford, StabilizerState,state_fidelity
from qiskit.visualization import array_to_latex
from qiskit_aer import AerSimulator
import random
from visualisation import LiveVisualizer,GraphWatcher,MatrixWatcher
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford
from qiskit_aer import AerSimulator

from classical_shadow_n_clifford import ClassicalShadow_N_CLIFFORD
from shadow_protocol import ShadowProtocol

In [ ]:
# Setup Experiment
class Id_Protocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 4
    
    def get_state_circuit(self) -> QuantumCircuit:
        circuit = QuantumCircuit(4)
        circuit.h(0)
        circuit.cx(0,1)
        return circuit

    def run_circuit_and_get_measurement(self, circuit) -> list[int]:
        sim = AerSimulator()

        # Run with 997 shots
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits= max(counts, key=counts.get)
        bit_list =  [int(bit) for bit in list(max_hits)]
        return bit_list[::-1] 
    
id_protocol = Id_Protocol()
# Create Classical Shadow Instance
shadow = ClassicalShadow_N_CLIFFORD(id_protocol)


In [ ]:
circuit_parallel = QuantumCircuit(4)
circuit_parallel.h(0)
circuit_parallel.cx(0,1)
cliff_parallel = Clifford(circuit_parallel)

This circuit creates the Bell state $|\Phi^+\rangle$. Since this is exactly our target state, the inner product (overlap) is 1.Calculation:The circuit produces:$$|\psi\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$$Overlap with target:$$\langle \Phi^+ | \psi \rangle = 1$$

In [ ]:
circuit_ortho = QuantumCircuit(4)
circuit_ortho.h(0)
circuit_ortho.cx(0,1)
circuit_ortho.z(0)
cliff_ortho = Clifford(circuit_ortho)

This circuit creates the state $|\Phi^-\rangle$. The $Z$-gate rotates the phase of the $|1\rangle$ component. This state is perpendicular (orthogonal) to our target, so the overlap is 0.Calculation:The $Z$-gate flips the sign:$$|\psi\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}}$$Overlap with target:$$\text{Overlap} = \frac{1}{2} (1 - 1) = 0$$

In [ ]:
circuit_scalar = QuantumCircuit(4)
cliff_scalar = Clifford(circuit_scalar)

To create a Clifford state with exactly 0.5 fidelity, we simply use the ground state $|00\rangle$. Since the target Bell state is an equal superposition of $|00\rangle$ and $|11\rangle$, choosing one of them gives exactly half the overlap.Calculation:The target state is $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$.Our state is $|\psi\rangle = |00\rangle$.$$\langle \Phi^+ | \psi \rangle = \frac{1}{\sqrt{2}} (\langle 00| + \langle 11|) |00\rangle = \frac{1}{\sqrt{2}}$$The Fidelity is the absolute square of the overlap:$$\text{Fidelity} = \left| \frac{1}{\sqrt{2}} \right|^2 = 0.5$$

In [ ]:
vis = LiveVisualizer() # crates the object but nothing will be displayed for the moment

target_density_matrix = shadow.get_original_density_matrix()

fidelity_history_parallel = []
fidelity_history_ortho = []
fidelity_history_scalar = []

while True:

    shadow.add_snapshot()
    
    shadow_size = shadow.get_shadow_size()

    actual_density_matrix = shadow.get_density_matrix_from_cliffords()

    
    fidelity_history_parallel.append((shadow_size, shadow.calculate_fidelity(cliff_parallel)))

    fidelity_history_ortho.append((shadow_size, shadow.calculate_fidelity(cliff_ortho)))

    fidelity_history_scalar.append((shadow_size, shadow.calculate_fidelity(cliff_scalar)))

    
    live_matrix_view = MatrixWatcher(
    title_template=f"Live Reconstruction (Shadow size {shadow_size})",
    matrix = actual_density_matrix
    )
    target_matrix_view = MatrixWatcher(
    title_template="Target State (Correct)",
    matrix = target_density_matrix
    )
    fidelity_graph_parallel = GraphWatcher(
    title="Fidelity Convergence parallel state", 
    target_value=1, 
    history=fidelity_history_parallel,
    y_min=0.0, 
    y_max=1.0
    )

    fidelity_graph_ortho = GraphWatcher(
    title="Fidelity Convergence orthognoal state", 
    target_value=0, 
    history=fidelity_history_ortho,
    y_min=0.0, 
    y_max=1.0
    )

    fidelity_graph_scalar = GraphWatcher(
    title="Fidelity Convergence with overlap of 0.5", 
    target_value=0.5, 
    history=fidelity_history_scalar,
    y_min=0.0, 
    y_max=1.0
    )


    vis.update(
        live_matrix_view,    
        target_matrix_view,
        fidelity_graph_parallel,
        fidelity_graph_ortho,
        fidelity_graph_scalar
    )